# Rf 9 0 1_Numintconfig
Numeric algorithm tuning: configuration and customization of how numeric (partial) integrals are executed



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:05 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"
#include "RooNumIntConfig.h"
#include "RooLandau.h"
#include "RooArgSet.h"
#include <iomanip>

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Adjust global 1d integration precision
 ----------------------------------------------------------------------------

Print current global default configuration for numeric integration strategies

In [3]:
RooAbsReal::defaultIntegratorConfig()->Print("v");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

Requested precision: 1e-07 absolute, 1e-07 relative

1-D integration method: RooIntegrator1D (RooImproperIntegrator1D if open-ended)
2-D integration method: RooAdaptiveIntegratorND (N/A if open-ended)
N-D integration method: RooAdaptiveIntegratorND (N/A if open-ended)

Available integration methods:

*** RooBinIntegrator ***
Capabilities: [1-D] [2-D] [N-D] 
Configuration: 
  1)  numBins = 100

*** RooIntegrator1D ***
Capabilities: [1-D] 
Configuration: 
  1)        sumRule = Trapezoid(idx = 0)

  2)  extrapolation = Wynn-Epsilon(idx = 1)

  3)       maxSteps = 20
  4)       minSteps = 999
  5)       fixSteps = 0

*** RooIntegrator2D ***
Capabilities: [2-D] 
Configuration: 
(Depends on 'RooIntegrator1D')

*** RooSegmentedIntegrator1D ***
Ca

Example: change global precision for 1d integrals from 1e-7 to 1e-6

 The relative epsilon (change as fraction of current best integral estimate) and
 absolute epsilon (absolute change w.r.t last best integral estimate) can be specified
 separately. For most p.d.f integrals the relative change criterium is the most important,
 however for certain non-p.d.f functions that integrate out to zero a separate absolute
 change criterium is necessary to declare convergence of the integral

 NB: This change is for illustration only. In general the precision should be at least 1e-7
 for normalization integrals for MINUIT to succeed.

In [4]:
RooAbsReal::defaultIntegratorConfig()->setEpsAbs(1e-6);
RooAbsReal::defaultIntegratorConfig()->setEpsRel(1e-6);

Numeric integration of landau pdf
 ------------------------------------------------------------------

Construct p.d.f without support for analytical integrator for demonstration purposes

In [5]:
RooRealVar x("x", "x", -10, 10);
RooLandau landau("landau", "landau", x, RooConst(0), RooConst(0.1));

Activate debug-level messages for topic integration to be able to follow actions below

In [6]:
RooMsgService::instance().addStream(DEBUG, Topic(Integration));

Calculate integral over landau with default choice of numeric integrator

In [7]:
RooAbsReal *intLandau = landau.createIntegral(x);
Double_t val = intLandau->getVal();
cout << " [1] int_dx landau(x) = " << setprecision(15) << val << endl;

[#2] INFO:Integration -- RooRealIntegral::ctor(landau_Int[x]) Constructing integral of function landau over observables(x) with normalization () with range identifier <none>
[#2] DEBUG:Integration -- landau: Adding observable x of server x as shape dependent
[#2] INFO:Integration -- landau: Observable x is suitable for analytical integration (if supported by p.d.f)
[#2] INFO:Integration -- landau: Observables (x) are numerically integrated
[#1] INFO:NumericIntegration -- RooRealIntegral::init(landau_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
 [1] int_dx landau(x) = 0.0989653362054419


Same with custom configuration
 -----------------------------------------------------------

Construct a custom configuration which uses the adaptive gauss-kronrod technique
 for closed 1D integrals

In [8]:
RooNumIntConfig customConfig(*RooAbsReal::defaultIntegratorConfig());
customConfig.method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");

Calculate integral over landau with custom integral specification

In [9]:
RooAbsReal *intLandau2 = landau.createIntegral(x, NumIntConfig(customConfig));
Double_t val2 = intLandau2->getVal();
cout << " [2] int_dx landau(x) = " << val2 << endl;

[#2] INFO:Integration -- RooRealIntegral::ctor(landau_Int[x]) Constructing integral of function landau over observables(x) with normalization () with range identifier <none>
[#2] DEBUG:Integration -- landau: Adding observable x of server x as shape dependent
[#2] INFO:Integration -- landau: Observable x is suitable for analytical integration (if supported by p.d.f)
[#2] INFO:Integration -- landau: Observables (x) are numerically integrated
[#1] INFO:NumericIntegration -- RooRealIntegral::init(landau_Int[x]) using numeric integrator RooAdaptiveGaussKronrodIntegrator1D to calculate Int(x)
 [2] int_dx landau(x) = 0.098957102921895


Adjusting default config for a specific pdf
 -------------------------------------------------------------------------------------

Another possibility: associate custom numeric integration configuration as default for object 'landau'

In [10]:
landau.setIntegratorConfig(customConfig);

Calculate integral over landau custom numeric integrator specified as object default

In [11]:
RooAbsReal *intLandau3 = landau.createIntegral(x);
Double_t val3 = intLandau3->getVal();
cout << " [3] int_dx landau(x) = " << val3 << endl;

[#2] INFO:Integration -- RooRealIntegral::ctor(landau_Int[x]) Constructing integral of function landau over observables(x) with normalization () with range identifier <none>
[#2] DEBUG:Integration -- landau: Adding observable x of server x as shape dependent
[#2] INFO:Integration -- landau: Observable x is suitable for analytical integration (if supported by p.d.f)
[#2] INFO:Integration -- landau: Observables (x) are numerically integrated
[#1] INFO:NumericIntegration -- RooRealIntegral::init(landau_Int[x]) using numeric integrator RooAdaptiveGaussKronrodIntegrator1D to calculate Int(x)
 [3] int_dx landau(x) = 0.098957102921895


Another possibility: change global default for 1d numeric integration strategy on finite domains

In [12]:
RooAbsReal::defaultIntegratorConfig()->method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");

Adjusting parameters of a specific technique
 ---------------------------------------------------------------------------------------

Adjust maximum number of steps of roointegrator1d in the global default configuration

In [13]:
RooAbsReal::defaultIntegratorConfig()->getConfigSection("RooIntegrator1D").setRealValue("maxSteps", 30);

Example of how to change the parameters of a numeric integrator
 (Each config section is a RooArgSet with RooRealVars holding real-valued parameters
  and RooCategories holding parameters with a finite set of options)

In [14]:
customConfig.getConfigSection("RooAdaptiveGaussKronrodIntegrator1D").setRealValue("maxSeg", 50);
customConfig.getConfigSection("RooAdaptiveGaussKronrodIntegrator1D").setCatLabel("method", "15Points");

Example of how to print set of possible values for "method" category

In [15]:
customConfig.getConfigSection("RooAdaptiveGaussKronrodIntegrator1D").find("method")->Print("v");

--- RooAbsArg ---
  Value State: clean
  Shape State: clean
  Attributes: 
  Address: 0x7f4d54602890
  Clients: 
  Servers: 
  Proxies: 
--- RooAbsCategory ---
  Value is "15Points" (1)
  Has the following possible values:
        WynnEpsilon = 0
        15Points = 1
        21Points = 2
        31Points = 3
        41Points = 4
        51Points = 5
        61Points = 6
